# 恢复已提取的1498个BGE Embeddings

这个notebook用来检查和恢复你已经提取的1498个embeddings数据

In [ ]:
import numpy as np
import os
from datetime import datetime

print("Libraries loaded successfully")

## Step 1: 检查temp文件

In [ ]:
temp_file = 'bge_embeddings_2k_llama.npy.temp.npy'

print("="*80)
print("检查临时文件")
print("="*80)

if not os.path.exists(temp_file):
    print(f"✗ 文件不存在: {temp_file}")
else:
    print(f"✓ 文件存在: {temp_file}")
    print(f"  文件大小: {os.path.getsize(temp_file) / 1024 / 1024:.2f} MB")
    print(f"  修改时间: {datetime.fromtimestamp(os.path.getmtime(temp_file))}")

## Step 2: 加载并分析数据

In [ ]:
print("正在加载embeddings...")
embeddings = np.load(temp_file)

print(f"\n✓ Embeddings加载成功!")
print(f"  Shape: {embeddings.shape}")
print(f"  数据类型: {embeddings.dtype}")
print(f"  内存占用: {embeddings.nbytes / 1024 / 1024:.2f} MB")

In [ ]:
# 检查有效数据
print("\n分析数据质量...")
vector_norms = np.linalg.norm(embeddings, axis=1)
valid_count = np.sum(vector_norms > 0.01)  # 非零向量
invalid_count = len(embeddings) - valid_count

print(f"  总样本数: {len(embeddings)}")
print(f"  ✓ 有效向量: {valid_count} ({valid_count/len(embeddings)*100:.1f}%)")
print(f"  ✗ 零向量: {invalid_count} ({invalid_count/len(embeddings)*100:.1f}%)")

print(f"\n向量范数统计:")
print(f"  最小值: {vector_norms.min():.4f}")
print(f"  最大值: {vector_norms.max():.4f}")
print(f"  平均值: {vector_norms.mean():.4f}")
print(f"  标准差: {vector_norms.std():.4f}")

In [ ]:
# 显示样本详情
print("\n前20个样本检查:")
for i in range(min(20, len(embeddings))):
    norm = vector_norms[i]
    status = "✓" if norm > 0.01 else "✗"
    print(f"  {status} 样本 {i:4d}: norm={norm:8.4f}")

if len(embeddings) > 20:
    print(f"\n最后20个样本检查:")
    for i in range(max(0, len(embeddings)-20), len(embeddings)):
        norm = vector_norms[i]
        status = "✓" if norm > 0.01 else "✗"
        print(f"  {status} 样本 {i:4d}: norm={norm:8.4f}")

## Step 3: 找出连续有效数据的范围

In [ ]:
# 找到第一个零向量的位置
is_valid = vector_norms > 0.01

# 找到连续有效数据的长度
consecutive_valid = 0
for i, valid in enumerate(is_valid):
    if valid:
        consecutive_valid = i + 1
    else:
        break

print("="*80)
print("数据完整性分析")
print("="*80)
print(f"\n连续有效数据: 前 {consecutive_valid} 个样本")
print(f"总有效数据: {valid_count} 个样本")

if consecutive_valid < valid_count:
    print(f"\n⚠️  警告: 数据中间有空洞（零向量）")
    print(f"   建议使用前 {consecutive_valid} 个连续数据")
else:
    print(f"\n✓ 数据连续，无空洞")

## Step 4: 保存恢复的数据

In [ ]:
# 保存有效的embeddings
if consecutive_valid > 0:
    recovered_embeddings = embeddings[:consecutive_valid]
    
    output_file = f'bge_embeddings_{consecutive_valid}_recovered.npy'
    np.save(output_file, recovered_embeddings)
    
    print("="*80)
    print("保存恢复的数据")
    print("="*80)
    print(f"\n✓ 已保存 {consecutive_valid} 个有效embeddings")
    print(f"  输出文件: {output_file}")
    print(f"  Shape: {recovered_embeddings.shape}")
    print(f"  文件大小: {os.path.getsize(output_file) / 1024 / 1024:.2f} MB")
    
    print(f"\n下一步操作:")
    print(f"  1. 你已经有了 {consecutive_valid} 个样本的embeddings ✓")
    print(f"  2. 如果需要2000个样本，还需提取 {2000 - consecutive_valid} 个")
    print(f"  3. 可以修改提取脚本，从第 {consecutive_valid} 个样本开始继续提取")
else:
    print("\n✗ 没有找到有效数据，无法恢复")

## 总结

运行完成后：
- 你的1498个embeddings数据已经安全保存 ✓
- 文件名: `bge_embeddings_XXXX_recovered.npy`
- 如需继续提取剩余样本，从saved位置开始即可